In [2]:
from torchtext import data, datasets
import torch
import os

In [3]:
# preserves case of words
inputs = data.Field(lower='preserve-case')

# No tokenization applied because the data is not seq
# unk_token=None: ignore out of vocabulary tokens, since these are grades
answers = data.Field(sequential=False, unk_token=None) # y: floats

# fine_grained=False - use the following grade mapping { 0,1 -> negativ; 2 -> neutral; 3,4 -> positive }
# filter=... - remove the neutral class to reduce the problem to binary classification
# train_subtrees=False - Use only complete review instead of also using subsentences (subtrees)
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained = False, train_subtrees = True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

/home/mbulte/anaconda2/lib/python2.7/site-packages/nltk/tree.py:619: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif token == close_b:
/home/mbulte/anaconda2/lib/python2.7/site-packages/nltk/tree.py:612: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if token[0] == open_b:


In [4]:
# build the initial vocabulary from the SST dataset
inputs.build_vocab(train, dev, test)

In [5]:
# then enhance it with the pre-trained glove model 
inputs.vocab.load_vectors('glove.6B.300d')

100%|██████████| 400000/400000 [01:09<00:00, 5752.51it/s]


In [6]:
# build the vocab for the labels (only consists of 'positive','negative')
answers.build_vocab(train)

In [ ]:
# You can use these iterators to train/test/validate the network :)
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)